In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from autograd import elementwise_grad, grad
import autograd.numpy as np
import scipy
from scipy.optimize import minimize

In [ ]:
n = 1

In [ ]:
betas = np.random.beta(5,1,size=100)

In [ ]:
sns.distplot(betas)

In [ ]:
def sal(par,y):
    assert len(par)>=4, "not enough params"
    return par[0]+par[1]*np.sinh(par[2]*np.arcsinh(y) + par[3])

In [ ]:
def sal_comp(par,y,n):
    assert len(par)>=4*n, "not enough params"
    comp = y
    for i in range(1,n):
        comp = sal(par[4*i:],comp)
    return comp

In [ ]:
# http://stackoverflow.com/questions/45599524/partial-derivative-using-autograd

In [ ]:
d_sal = elementwise_grad(sal_comp,1)

In [ ]:
d_sal

In [ ]:
def cov_kernel(gamma,x):
    return np.exp(-gamma*np.abs(x))

In [ ]:
def nllf(par):
    phi_y = sal_comp(par,betas,n)
    phi_y = phi_y[np.newaxis]
    t_phi_y = np.transpose(phi_y)
    
    cov_xx = cov_kernel(1,np.log(np.exp(t_phi_y)) @ np.exp(-(phi_y)))
    gaussian_params = 0.5 * (phi_y) @ np.linalg.inv(cov_xx) @ t_phi_y
    
    print(np.linalg.det(cov_xx))
    
    return  0.5 * np.log(np.linalg.det(cov_xx)) + gaussian_params - sum(np.log(d_sal(par,betas,n)))

In [ ]:
res = minimize(nllf, np.random.rand(4*n), method='nelder-mead')
while res.success == False:
    try:
        res = minimize(nllf, np.random.rand(4*n), method='SLSQP')
        print(np.random.rand(4*n))
    except:
        pass

In [ ]:
res

In [ ]:
res.x

In [ ]:
transformed_betas = sal_comp(res.x,betas,n)

In [ ]:
sns.distplot(transformed_betas)